# **机器学习公开课 - 机器学习的未来：AutoML与超参数优化**
## Day 3 贝叶斯优化原理与HyperOpt

> 节选自《2021机器学习实战》正课<br>
> 作者：@菜菜TsaiTsai<br>
> 版本号：2021/12/06 V1<br>

## 0 课程规划

欢迎来到为期三天的《机器学习的未来：AutoML与超参数优化》公开课。在这门课程中，我将用三天时间带你AutoML当前研究现状，并深入了解超参数优化算法中的网格优化与贝叶斯优化。当你完成这门课程时，你将能够自由应用数种网格搜索算法与数种贝叶斯优化算法，同时你将会了解AutoML领域所面临的挑战和机遇。如果想系统地学习更多超参数优化的信息，则可以向客服小可爱咨询正课《2021机器学习实战》。

**DAY 1：AutoML前沿进展与枚举网格搜索**

**DAY 2：随机网格搜索与Halving网格搜索**

**DAY 3：贝叶斯优化与HyperOpt**

为掌握这门课程，你需要先掌握：
- Python基础 >> 《九天老师的Python基础》<br>https://www.bilibili.com/video/BV1U54y1W7jw?spm_id_from=333.999.0.0

- 机器学习基础 >> 《2021机器学习实战公开课》<br>包括超参数、泛化能力等基本概念<br>交叉验证等基本工具<br>随机森林等基本算法<br>https://www.bilibili.com/video/BV1f3411r7EP?share_source=copy_web

In [1]:
import re
import sklearn
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt
import time
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV

In [2]:
for package in [sklearn,mlp,np,pd]:
    print(re.findall("([^']*)",str(package))[2],package.__version__)

sklearn 1.0.1
matplotlib 3.4.3
numpy 1.20.3
pandas 1.3.4


In [ ]:
#pip install --upgrade scikit-learn
#conda update scikit-learn

## 1 贝叶斯优化（Bayesian Optimization）基础方法

- 贝叶斯优化的基本原理

在之前的课程中我们讲解了网格搜索、随机网格搜索与Halving网格搜索，无论具体每种网格搜索的思想如何变化，网格优化都是在一个大参数空间中、尽量对所有点进行验证后再返回最优损失函数值的方法，这一类方法再计算量与计算时间上有着不可避免的缺陷，因此才会有随机、Halving等试图缩短训练时间、让整体网格搜索更加适合于大型数据和大型空间的手段。然而，尽管sklearn在提高网格搜索效率方面做出了种种优化，但上述方法仍然无法在效率和精度上做到双赢，若希望更快速的进行参数搜索、并且搜索出一组泛化能力尽可能强的参数，目前的常见做法还是选用一些带有**先验过程**的调参工具，即一些基于贝叶斯过程调参工具。

贝叶斯优化方法是当前超参数优化领域的SOTA手段，可以被认为是当前最为先进的优化框架，它可以被应用于AutoML的各大领域，不止限于超参数搜索HPO的领域，更是可以被用于神经网络架构搜索NAS以及元学习等先进的领域。现代几乎所有在效率和效果上取得优异成果的超参数优化方法都是基于贝叶斯优化的基本理念而形成的，因此贝叶斯优化是整个AutoML中学习的重点。

然而，虽然贝叶斯优化非常强大，但整体的学习难度却非常高。在学习贝叶斯优化之前，学习者不仅需要充分理解机器学习的主要概念和算法、熟悉典型的超参数优化流程，还需要对部分超出微积分、概率论和线性代数的数学知识有所掌握。特别的是，贝叶斯优化算法本身，与贝叶斯优化用于HPO的过程还有区别。在我们课程有限的时间内，我将重点带大家来看贝叶斯优化用于HPO的核心过程。

首先，我们不理会HPO的问题，先来看待下面的例子。假设现在我们知道一个函数$f(x)$以及其自变量$x$的定义域，并且我们知道函数$f(x)$是相对均匀、平稳的函数，并不会出现突然升高或突然下降的情况。现在，我们希望求解出$x$的取值范围上$f(x)$的最小值，你打算如何求解这个最小值呢？

面对这一问题，我们有一个通俗的解法，那就是将全域的$x$带入$f(x)$计算出所有$f(x)$可能的结果，一旦整个$f(x)$的结果被计算出来，那$f(x)$的最小值也就一目了然了。

然而，现在的函数$f(x)$是一个异常复杂的函数，计算量非常大、需要的时间也很长，同时我们的计算资源非常紧张，最多只能在函数$f(x)$上尝试20次左右，所有我们不太可能将全域$x$都带入进行计算，然后再求解最小值。于是我们选择在$x$的定义域上随机选择了4个点，并将4个点带入$f(x)$进行计算，得到了如下结果：

![01](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/01.png)

好了，现在有了这4个观测值，你能告诉我$f(x)$的最小值在哪里吗？你认为最小值点可能在哪里呢？大部分人会倾向于认为，最小值点在已观测出的$f(x)$值中，最小的那个值的附近，但也有许多人不这么认为。当我们有了4个观测值，并且知道我们的函数时相对均匀、平稳的函数，那我们可能对函数的整体分布有如下猜测：

![02](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/02.png)

当我们对函数整体分布有一个猜测时，这个分布上一定会存在该函数的最小值。同时，不同的人可能对函数的整体分布有不同的猜测，不同猜测下对应的最小值也是不同的。

![03](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/03.png)

![04](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/04.png)

现在，假设我们邀请了数万个人对该问题做出猜测，每个人所猜测的曲线如下图所示。不难发现，在观测点的附近，每个人猜测的函数值差距不大，但是在远离远侧点的地方，每个人猜测的函数值就高度不一致了。这也是当然的，因为观测点之间函数的分布如何完全是未知的，并且该分布离观测点越远时，我们越不确定真正的函数值在哪里，因此人们猜测的函数值的范围非常巨大。

![05](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/05.png)

现在，我们将所有猜测求均值，并将任意均值周围的潜在函数值所在的区域用色块表示，可以得到一条所有人猜测的平均曲线。不难发现，色块所覆盖的范围其实就是大家猜测的函数值的上界和下界，而任意$x$所对应的上下界差异越大，表示人们对函数上该位置的猜测值的越不确定。**因此上下界差异可以衡量人们对该观测点的置信度**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/06.png)

在观测点周围，置信度总是很高，远离观测点的地方，置信度总是很低，所以如果我们能够在置信度很低的地方补充一个实际的观测点，我们就可以很快将众人的猜测统一起来。当整个函数上的置信度都非常高时，我们可以说我们得出了一条与真实的$f(x)$曲线高度相似的曲线。并且，观测点越多，我们估计出的曲线会越接近真实的$f(x)$。然而，由于计算量有限，我们每次进行观测时都要非常谨慎地选择观测点。那现在，如何选择观测点才能够最大程度地帮助我们找到$f(x)$上的最小值呢？

有非常多的方法，其中最简单的手段是使用最小值出现的频数进行判断。由于不同的人对函数的整体分布有不同的猜测，不同猜测下对应的最小值也是不同的，根据每个人猜测的函数结果，我们在$X$轴上划将定义域区间进行均匀的划分，如果有某个猜测的最小值落在其中一个区间中，我们就对该区间进行计数。当有数万个人进行猜测之后，我们同时也绘制了基于$X$轴上不同区间的频数图，频数越高，说明猜测最小值在该区间内的人越多，反之则说明该猜测最小值在该区间内的人越少。

当我们绘制出频数图之后，很明显，我们知道最小值最有可能在的区间就在x=0.7左右的位置。当我们不取新的观测点时，现在$f(x)$上可以获得的可靠的最小值就是x=0.6时的点，但我们如果在x=0.7处取新的观测值，我们就很有可能找到比当前x=0.6的点还要小的$f_min$。因此，我们可以就此决定，在x=0.7处进行观测。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/07.png)

当我们在x=0.7处取出观测值之后，我们就有了5个已知的观测点。现在，我们再让数万人根据5个已知的观测点对整体函数分布进行猜测，猜测完毕之后再计算当前最小值频数最高的区间，然后再取新的观测点对$f(x)$进行计算。当允许的计算次数被用完之后（比如，20次），整个估计也就停止了。

你发现了吗？在这个过程当中，我们其实在不断地优化我们对目标函数$f(x)$的假设，虽然没有对$f(x)$进行全部定义域上的计算，也没有找到最终确定一定是$f(x)$分布的曲线，但是随着我们观测的点越来越多，我们对函数的估计是越来越准确的，因此也有越来越大的可能性可以估计出$f(x)$真正的最小值。**这个优化的过程，就是贝叶斯优化**。

![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/08.gif)

在贝叶斯优化的数学过程当中，我们主要执行以下几个步骤：

- 1 定义需要估计的$f(x)$以及$x$的定义域

- 2 取出有限的n个$x$上的值，求解出这些$x$对应的$f(x)$（求解观测值）

- 3 根据有限的观测值，对函数分布进行假设（该假设被称为贝叶斯优化中的先验知识），得出该假设分布上的目标值

- 4 定义某种规则，确定下一个需要计算的观测点

并持续在2-4步骤中进行循环，直到假设分布上的目标值达到我们的标准，或者所有计算资源被用完为止。以上流程又被称为序贯模型优化（SMBO），是最为经典的贝叶斯优化方法。

在实际的运算过程当中，尤其是超参数优化的过程当中：

- 需要定义的$f(x)$一般是交叉验证的结果/损失函数的结果，而$x$就是超参数空间，$x$的定义域中是超参数的各种组合

- 有限的观测值数量是贝叶斯优化的超参数之一，该观测数量也决定了整个贝叶斯优化的迭代次数

- 在第3步根据有限的观测值、对函数分布进行假设的工具被称为**概率代理模型**，概率代理模型往往是一些强大的算法，最常见的比如高斯过程、随机森林等等。传统数学推导中往往使用高斯过程，但现在最先进、最普及的优化库中默认是使用基于随机森林的TPE过程。

- 在第4步中用来确定下一个观测点的规则被称为**采集函数**，最常见的主要是概率增量（比如我们计算的频数）、期望增量、信息熵等等，其中大部分优化库中默认使用期望增量，具体表达式如下：

$$E[max(f_{min} - f(x_n),0)]$$

其中$f_{min}$是当前最小的$f(x)$观测值，$f(x_n)$是第n次迭代中观测点的观测值。

如果当前最小观测值就是本次迭代中的观测值或小于本次迭代中的观测值，则说明本次取出的结果还不如之前的观测点好，期望增量=0，函数最小值不在$f(x_n)$附近，需要重新选取距离$x_n$较远的新观测点。

如果当前最小观测值$f_{min}$大于本次迭代中的观测值$f(x_n)$，则说明$f(x_n)$的选取是成功的，应当更新当前最小值$f_{min}$，并继续围绕在$x_n$附近寻找下一个观测值。

经过整体优化过程之中，我们要找到的是估计出的$f(x)$上的最小值所对应的$x$，也就是估计出的交叉验证/损失函数最小值所对应的参数组合。

对于贝叶斯优化，真正的难点有以下几个：

- 1 高斯过程或随机森林是如何根据几个很少的观测点估计出整体函数曲线的？

- 2 采集函数的结果在代码/数学上又是如何直接影响观测点的选择的？

- 3 我们怎么知道迭代几次才能够让估计出的函数曲线高度接近真正的$f(x)$呢？

而每一个问题展开之后都是一个很难的数学问题。遗憾的是，在直播课当中，我们没有足够的时间去解释每一个点，但在正式课程当中，我们将展开讲解高斯过程的详细内容。

### **<font color="red">贝叶斯优化应用代码将在正课中呈现</font>**
### **<font color="red">贝叶斯优化数学推导，概率代理模型/采集函数详解在正课中呈现</font>**